### Notebook to make a table to track hyperparameter tuning

In [1]:
import pandas as pd
import numpy as np
import itertools

Designate parameters to test

In [2]:
# metric loss-related parameters
metric_loss_type = ["NT-Xent", "triplet"] # type of metric loss function. In theory NT-Xent should do better, but in practice it has been triplet
margin = [0.1, 1, 2, 5] # sets tolerance for metric loss. The larger it is, the longer the range over which metric loss will be applied
                        # operates in z-score units    
metric_weight = [0, 10, 25, 50] # weight of metric loss relative to KLD and recon loss scores
self_target_prob = [0.25, 0.5, 0.75] # fraction of metric comparisons that are within a single trajectory
temperature = [0.1] # scaling param. Only used for NT-Xent
time_only_flag = [0, 1] # if 1, use only time info for metric comparisons

# general model params
learning_rate = [1e-4] #[1e-3, 5e-4, 1e-4]
latent_dim = [100] #[10, 25, 50, 100]
beta = [0.1, 1, 10] # weight applied to KLD loss
# loss = recon_loss (~200) + beta*KLD_loss (~15) + 10*metric_loss (~5)
batch_size = [2048]
zn_frac = [0.2] # fraction of latent variables that are used to capture non-biological variability

holdout_flag = [0, 1] # If 1, hold out selected perturbation types

In [3]:
# Create a dictionary of parameter names and their values
param_grid = {
    'metric_loss_type': metric_loss_type,
    'margin': margin,
    'metric_weight': metric_weight,
    'self_target_prob': self_target_prob,
    'time_only_flag': time_only_flag,
    'temperature': temperature,
    'learning_rate': learning_rate,
    'latent_dim': latent_dim,
    'beta': beta,
    'batch_size': batch_size,
    'zn_frac': zn_frac,
    'holdout_flag': holdout_flag
}

# Generate all combinations
all_combinations = list(itertools.product(*param_grid.values()))

# Convert to a DataFrame
hyperparam_df = pd.DataFrame(all_combinations, columns=param_grid.keys())

hyperparam_df.shape

(1152, 12)

This looks great...but we have over 9,200 combinations to test. Given the computational resources available,  this would take something like: (9200 models) x (150 epochs) x (0.025 hr per epoch) / (4 GPUs) = 6624 hours (i.e. about a year!!)

In [4]:
# Remove all but one of the cases with no metric weight (since rest of parameters don't matter)
z_indices0 = np.where((hyperparam_df["metric_weight"]==0) & (hyperparam_df["holdout_flag"]==0))[0]
z_indices1 = np.where((hyperparam_df["metric_weight"]==0) & (hyperparam_df["holdout_flag"]==1))[0]

keep_indices = [i for i in list(hyperparam_df.index) if i not in z_indices0]
keep_indices = [i for i in keep_indices if i not in z_indices1]

keep_indices.append(z_indices0[0])
keep_indices.append(z_indices1[0])

keep_indices = np.asarray(keep_indices)
hyperparam_df = hyperparam_df.loc[keep_indices, :]
hyperparam_df.reset_index(inplace=True, drop=True)
hyperparam_df.shape

(866, 12)

In [5]:
# limit binary flag to only co-occur with self_target_prob=0.5
t_filter = ~((hyperparam_df["time_only_flag"]==1) & (hyperparam_df["self_target_prob"]!=0.5))

hyperparam_df = hyperparam_df.loc[t_filter, :]
hyperparam_df.reset_index(inplace=True, drop=True)
hyperparam_df.shape

(578, 12)

In [6]:
import os

# add col to save path info
hyperparam_df["model_path"] = ""
hyperparam_df["completed"] = 0

# randomly assign processing order
df_indices = list(hyperparam_df.index)
np.random.seed(301)
process_id = np.random.choice(df_indices, len(df_indices), replace=False)
hyperparam_df["process_id"] = process_id

outdir = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/metadata/"
hyperparam_df.to_csv(os.path.join(outdir, "hyperparam_sweep01_df.csv"), index=False)